In [2]:
import numpy as np
from numpy import pi
from numpy.linalg import eig

In [1]:
def AIC(snapshots, M, eigenValues):
#---M: the number of array elements-------------# 
#---eigenValues: eigen values with descendant sort--------#
    t = float('inf')
    for n in range(M - 1):
        num = 0
        den = 1
        for i in range(n, M):
            num += eigenValues[i]
            den *= eigenValues[i]
        Lambda = (num/(M - n)) / np.power(den, (1/(M - n)))
        entropy = 2*snapshots*(M - n)*np.log(Lambda) + 2*n*(2*M - n)
        if entropy > t:
            return n - 1
        else:
            t = entropy

In [2]:
def STD_MUSIC(SNR = 20, n = 200, M = 8):
#----------SNR: signal noise radio, n: the number of snapshots, M: the number of array.-------#
#----------Consider a ULA, where the array sapcing is a half wavelength of the signal.--------#
    c = 3e8
    f = 2.4e9
    lamb = c/f
    d = lamb/2

    #---------Sample: the number of snapshots = n = 200-------------------------------------------#
    fs = 3*f
    Ts = 1/fs
    Ns = Ts*np.arange(n)

#     print('SNR:', SNR)
#     print('Snapshots:', n)
#     print('The number of sensors:', M)
    
#----------Consider noises are generated from a zero mean Gaussian distribution.--------------#
    sigma_N = 0.1
    R_N = np.diag(sigma_N * np.ones(M))    
#     print('Covariance matrix of noise:\n')
#     print(R_N)
    
    N = np.random.multivariate_normal(np.zeros(R_N.shape[0]), R_N, n)
    phase = np.mat([np.exp(-1j*2*pi*f*Ns + np.random.rand())])
    N = np.multiply(N, phase.T)                    # N is noisy matrix generated from a Gaussian 
#     print('Size of Noisy matrix(Each row is a sample):\n')
#     print(N.shape)                                 # Each row is a sample
    
    #----------Consider three uncorrlated sources at -10 degree, 0 degree and 10 degree,----------------------#
    #----------Each source is generated from a zerom mean Gaussian distribution.------------------#    
    theta_S = np.array([-10, 0, 10])
    sigma_S = sigma_N * np.power(10, (SNR/20))
    R_S = np.diag(sigma_S * np.ones(len(theta_S)))
#     print('Covariance matrix of noise:\n')
#     print(R_S)
    
    S = np.random.multivariate_normal(np.zeros(R_S.shape[0]), R_S, n)
    S = np.multiply(S, phase.T)                    # S is signal matrix generated from a Gaussian 
#     print('Size of Signal matrix(Each row is a sample):\n')
#     print(S.shape)                                 # Each row is a sample
    
    x_k = d * np.arange(M)
    A = np.zeros((M, theta_S.size), dtype=complex)
    for col in range(A.shape[1]):
        A[:, col] = np.mat([np.exp(-1j*2*pi*f*(np.dot(x_k, np.sin(np.deg2rad(theta_S[col]))) / c))], dtype=complex)

#     print('Manifold matrix:\n')
#     print('Size:', A.shape, '\n')
#     print(A)
    
    #---------X = AS + N--------------------------------------------------------------------------#
    X = np.dot(A, S.T) + N.T
#     print('X size:', X.shape)
#     print('Each column is a sample')
    
    R_h = np.dot(X, X.H) / n
    
    D, V = eig(R_h)
    D_index = np.argsort(-D)
    numOfSources = len(theta_S)
#     print(numOfSources)
    V = V[:, D_index]
    Us = V[:, np.arange(numOfSources)]
    
    theta = np.arange(-90, 91, 0.1)
    P_mu = np.zeros(len(theta), dtype=complex)
    for ite in range(len(theta)):
        a = np.mat(np.exp(-1j*2*pi*f*(x_k*np.sin(np.deg2rad(theta[ite]))/c)))
        P_mu[ite] = 1/(a*(np.eye(Us.shape[0]) - Us*Us.H)*a.H)
    
    return theta, P_mu

In [1]:
def SS_MUSIC(SNR = 20, n = 200, M = 8):
#----------SNR: signal noise radio, n: the number of snapshots, M: the number of array.-------#
#----------Consider a ULA, where the array sapcing is a half wavelength of the signal.--------#
    c = 3e8
    f = 2.4e9
    lamb = c/f
    d = lamb/2

#---------Sample: the number of snapshots = n = 200-------------------------------------------#
    fs = 3*f
    Ts = 1/fs
    Ns = Ts*np.arange(n)

#     print('SNR:', SNR)
#     print('Snapshots:', n)
#     print('The number of sensors:', M)

#----------Consider noises are generated from a zero mean Gaussian distribution.--------------#
    sigma_N = 0.1
    R_N = np.diag(sigma_N * np.ones(M))    
#     print('Covariance matrix of noise:\n')
#     print(R_N)

    N = np.random.multivariate_normal(np.zeros(R_N.shape[0]), R_N, n)
    phase = np.mat([np.exp(-1j*2*pi*f*Ns + np.random.rand())])
    N = np.multiply(N, phase.T)                    # N is noisy matrix generated from a Gaussian

#----------Consider three sources at -10 degree, 0 degree and 10 degree.----------------------#
#----------Sources at -10 degree and 0 degree is coherent.------------------------------------#
#----------Each source is generated from a zerom mean Gaussian distribution.------------------#
    theta_S = np.array([-10, 0, 10])
    sigma_S = sigma_N * np.power(10, (SNR/20))
    R_S = np.mat([[sigma_S, 0.99*sigma_S, 0], [0.99*sigma_S, sigma_S, 0], [0, 0, sigma_S]])

    S = np.random.multivariate_normal(np.zeros(R_S.shape[0]), R_S, n)
    S = np.multiply(S, phase.T)                    # S is signal matrix generated from a Gaussian 

    x_k = d * np.arange(M)
    A = np.zeros((M, theta_S.size), dtype=complex)
    for col in range(A.shape[1]):
        A[:, col] = np.mat([np.exp(-1j*2*pi*f*(np.dot(x_k, np.sin(np.deg2rad(theta_S[col]))) / c))], dtype=complex)

#     print('Each row is a sample')
#     print('Signal matrix S size:', S.shape)
#     print('Noise matrix N Size:', N.shape)
#     print('Manifold matrix A size:', A.shape)

    X = np.dot(A, S.T) + N.T

    R_h = np.dot(X, X.H) / n                        # R hat: covariance matrix of samples

    m = round((M + (len(theta_S)/2) + 1)/2)                    # m S.T. D <= m <= M - D/2 + 1
    p = M + 1 - m
    R_fk = np.mat(np.zeros((m, m)))
    R_bk = np.mat(np.zeros((m, m)))

    for k in range(1, p + 1):
        Z_k = np.mat(np.hstack((np.zeros((m, k - 1)), np.eye(m), np.zeros((m, p - k)))))
        R_fk = R_fk + Z_k*R_h*Z_k.H

        Q_k = np.mat(np.hstack((np.zeros((m, k - 1)), np.fliplr(np.eye(m)), np.zeros((m, p - k)))))
        R_bk = R_bk + Q_k*np.conj(R_h)*Q_k.H

    R_f = R_fk/p
    R_b = R_bk/p
    R_fb = (R_f + R_b)/2

    D, V = eig(R_fb)
    D_index = np.argsort(D)
    numOfSources = len(theta_S)
    V = V[:, D_index]
    U_n = V[:, np.arange(m - numOfSources)]

    theta = np.arange(-90, 91, 0.1)
    P_mu = np.zeros(len(theta), dtype=complex)
    for ite in range(len(theta)):
        a = np.mat(np.exp(-1j*2*pi*f*(d*np.arange(m)*np.sin(np.deg2rad(theta[ite]))/c)))
        P_mu[ite] = 1/(a*U_n*U_n.H*a.H)
    
    return theta, P_mu